# Kachra Rani 
## Team:
### 1. Syed Hassan Haider
### 2. Inara Naseem G H Chagani 
### 3. Muhammad Shahrom Ali 

In [1]:
# Input: Image 
# Output : Label for that Image 

# Steps: 
# 1. Loading Image and Preprocessing 
# 2. Defining Model Architecture 
# 3. Training the Model 
# 4. Estimation of performance

###### Importing and Housekeeping stuff

In [2]:
#imports
import numpy as np 

import pickle

import cv2

from sklearn import svm
from skimage import color

from skimage.feature import hog
from sklearn.metrics import classification_report,accuracy_score

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.style.use('fivethirtyeight')

In [3]:
#FILENAMES FOR EXPORTED DATA 

FEATURE_DATA = 'KR_extracted_features'
RAW_DATASET = 'KR_dataset'
KR_MODEL = 'KR_model'

# Supporting information defined 
dataset_info = {
    # FORMAT::garbage_type : (no_of_imgs, label_value)
    'glass' : (501, 0),
    'plastic' : (482, 1),
    'cardboard' : (403, 2),
    'paper' : (594, 3),
    'metal' : (410, 4),
    'trash' : (137, 5)
}

#Label_value (int) : garbage_type (string)
labels = {dataset_info[i][1] : i for i in dataset_info}

###### Helper Functions 

In [4]:
#export files
def export(filename, to_export):
    np.save(filename + ".npy", to_export)

In [5]:
def load(filename):
    test_frame = np.load(filename + ".npy")

###### Load Data

In [16]:
def preprocess_img(img):
    
    scale_percent = 50 # percent of original size
    
    #Resize the image for less pixels 
    width = int(256)
    height = int(192)
    dim = (width, height) 
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

    #convert image to grayscale 
    img = color.rgb2gray(img)
    
    return img

In [7]:
def load_dataset():
    KR_dataset = list()

    for garbage_type in dataset_info:
    
        img_count = dataset_info[garbage_type][0]
        garbage_label = dataset_info[garbage_type][1]
    
        #LOAD IMAGES OF SELECTED GARBAGE TYPE
        for i in range(1,  img_count + 1):

            img_path = './Dataset/' + garbage_type + "/" + garbage_type + str(i) + '.jpg'

            img = preprocess_img( mpimg.imread( img_path ) ) 

            KR_dataset.append((img, garbage_label))

    #Loaded Dataset
    KR_dataset = np.array(KR_dataset, dtype=object)
    
    return KR_dataset

###### Feature Extraction 

In [8]:
def feature_extraction(img):
    ppc = 16 #PIXELS PER CEL 
    return hog(img, 
               orientations=8,
               pixels_per_cell=(ppc,ppc),
               cells_per_block=(4, 4),
               block_norm= 'L2',
               visualize=True)

In [9]:
def gen_feature_set(dataset):
    #Feature Extraction using Histogram of Gradient

    hog_images = []
    hog_features = []

    for image in dataset:

        feat_desc, hog_image = feature_extraction(image[0])

        hog_images.append(hog_image)
        hog_features.append(feat_desc)
    
    hog_features = np.array(hog_features)
    labels = np.array( [img[1] for img in dataset] ).reshape(len(dataset), 1) #make collumn vector

    # each row corresponds to hog_features of some img i
    # and at the end is the label of that image 
    data_frame = np.hstack((hog_features, labels))
    
    return data_frame

###### Splitting Data

In [10]:
def split_data(data_frame):
    
    percentage = 80
    partition = int(data_frame.shape[0]*percentage/100)
    
    x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
    
    y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
    
    return (x_train, y_train, x_test, y_test)

###### Generating Model

In [11]:
def KR_classifier(x_train, y_train):
    
    model_kr = svm.SVC()
    model_kr.fit(x_train,y_train)
    
    return model_kr

###### Predictions and Model Evaluation

In [12]:
def model_eval(y_test, y_pred):
    print("Accuracy: " + str(accuracy_score(y_test, y_pred) * 100) + "\%")
    print('\n')
    print(classification_report(y_test, y_pred))

###### Generate Model

In [13]:
# Load Dataset 
KR_dataset = load_dataset()

# Export Dataset
export(RAW_DATASET, KR_dataset)

# Shuffle Dataset
np.random.shuffle(KR_dataset)


# Extract Features and Create Feature Set
KR_data_frame = gen_feature_set(KR_dataset)

# Export Feature Set 
export(FEATURE_DATA, KR_data_frame)

# Split Feature Set 
x_train, y_train, x_test, y_test = split_data(KR_data_frame)


# Fit Model
KR_model = KR_classifier(x_train, y_train)

# Export model
export(KR_MODEL, KR_model)

#Test Model
y_pred = KR_model.predict(x_test)

# Metrics on Model
model_eval(y_test, y_pred)

Accuracy: 55.533596837944664\%


              precision    recall  f1-score   support

         0.0       0.47      0.60      0.53        90
         1.0       0.45      0.45      0.45       100
         2.0       0.76      0.65      0.71        84
         3.0       0.57      0.82      0.67       118
         4.0       0.61      0.34      0.44        88
         5.0       0.00      0.00      0.00        26

    accuracy                           0.56       506
   macro avg       0.48      0.48      0.47       506
weighted avg       0.54      0.56      0.53       506



c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###### Run Model

In [18]:
# Run model

#load image in img 
img_run = preprocess_img( mpimg.imread("./test_img.jpg") ) 

feature_descriptor = np.array([feature_extraction(img_run)[0],])
predicted_label = KR_model.predict(feature_descriptor)

print("The image contains", labels[predicted_label[0]])

The image contains plastic
